In [46]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# Original 3D IXI data from Transmorph repo
# This notebook converted it to a 2D dataset

In [35]:
import os, sys
import numpy as np
import scipy.ndimage
import torch
# from torchvision import transforms
from torch.utils.data import Dataset
from glob import glob
from os import path
import random

class NICE_Transeg_Dataset_Infer(Dataset):
    def __init__(self, data_path, device, transform=torch.from_numpy):
        self.transform = transform
        self.device = device
        self.images = []
        self.labels = []
        files = glob(path.join(data_path, "*.pkl"))
        self.files = files
        print(f"{data_path.split('/')[-1]} file num: {len(files)}")

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        image, label = np.load(self.files[idx], allow_pickle=True)
        return self.transform(image).unsqueeze(0).to(self.device), self.transform(label).float().unsqueeze(0).to(self.device), self.files[idx]
        # return torch.reshape(self.transform(image)[:,:,:144], (144, 192, 160)).unsqueeze(0).to(self.device), self.transform(label).unsqueeze(0).to(self.device)

def print_gpu_usage(note=""):
    print(f"{note}: %fGB"%(torch.cuda.memory_allocated(0)/1024/1024/1024), flush=True)

In [68]:

from torch.utils.data import DataLoader
import torch.nn as nn

train_dir = '/content/drive/MyDrive/IXI/Train'
atlas_dir = '/content/drive/MyDrive/IXI/Atlas'
test_dir = '/content/drive/MyDrive/IXI/Test'
val_dir = '/content/drive/MyDrive/IXI/Val'
device = 'cpu'
batch_size = 1

# test_dl = DataLoader(NICE_Transeg_Dataset_Infer(test_dir, device), batch_size=1, shuffle=False, drop_last=False)
# val_dl = DataLoader(NICE_Transeg_Dataset_Infer(val_dir, device), batch_size=1, shuffle=False, drop_last=False)
# train_dl = DataLoader(NICE_Transeg_Dataset_Infer(train_dir, device), batch_size=1, shuffle=False, drop_last=False)
atlas_dl = DataLoader(NICE_Transeg_Dataset_Infer(atlas_dir, device), batch_size=1, shuffle=False, drop_last=False)

Atlas file num: 5


In [47]:
x = np.load('/content/drive/MyDrive/NICE_Transeg/IXI2D/Train/data/0.npy')
print(x.shape)

(160, 224, 1)


In [69]:
%matplotlib inline
from matplotlib import pyplot as plt

id = 0

#train
for image, label, file_name in atlas_dl:
  image = image.squeeze()[:,100,:].unsqueeze(-1).numpy()
  label = label.squeeze()[:,100,:].unsqueeze(-1).numpy()

  np.save(path.join('/content/drive/MyDrive/NICE_Transeg/IXI2D/Atlas/data/', f'{id}.npy'), image)
  np.save(path.join('/content/drive/MyDrive/NICE_Transeg/IXI2D/Atlas/label/', f'{id}.npy'), label)
  id += 1

  # plt.imshow(image, interpolation='nearest')
  # plt.show()
  # plt.imshow(label, interpolation=)
  # plt.show()


In [72]:
# !tar -zcvf IXI2D.tar.gz /content/drive/MyDrive/NICE_Transeg/IXI2D
!mv /content/IXI2D.tar.gz /content/drive/MyDrive/NICE_Transeg/IXI2D.tar.gz